In [ ]:
from collections import Counter
import os
from custom.PC_Class import PC

PC_obj = PC()


def majority_element(lst, percentage):
    if not isinstance(lst, pd.Series):
        lst = pd.Series(lst)
    
    # Check if all entries are NaN or if the list is empty
    if lst.isna().all() or lst.empty:
        return 'Non Existent'
    # Calculate the number of occurrences of each item in the list
    count = Counter(lst)
    # Find the most common item and its count
    most_common, count_most_common = count.most_common(1)[0]
    # Calculate the percentage of the most common item
    occurrence_percentage = (count_most_common / len(lst)) * 100
    # Compare with the given percentage threshold
    if occurrence_percentage >= percentage:
        return most_common
    else:
        return "Not Certain"
    
def first_item(lst):
    if not isinstance(lst, pd.Series):
        lst = pd.Series(lst)
    
    # Check if all entries are NaN or if the list is empty
    if lst.isna().all() or lst.empty:
        return 'Non Existent'
    
    # Return the first item in the list
    return int(lst.iloc[0])

def string_to_int_list(s):
    if type(s) == list:
        return s
    return [int(item) for item in s.replace("[", "").replace("]", "").split(',')]

# MSA normal

In [ ]:
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt

direct = os.getcwd()
file_dir = direct + "\\data\\vos-instances\\Scenario_results\\"
files = os.listdir(file_dir)
dates = list(set([file.split("_")[1] for file in files if "Scenario_" in file]))

# Get final delivery solutions
df_free  = pd.read_csv('totals.csv')
df_free['SHIPMENTNUMBER'] = df_free['SHIPMENTNUMBER'].apply(string_to_int_list)
df_free = df_free.explode("SHIPMENTNUMBER")


file_path = direct + "\\data\\vos_input_data\\MultiHubData3_cleaned.csv" 
df_orders =  pd.read_csv(file_path)

routing_dict = {}

for dat in dates:
    print(dat)
    total_count = []
    volume_pallets = []
    total_not_certain = []
    total_certain = []
    total_certain_correct = []
    total_certain_incorrect = []

    volume_not_certain = []
    volume_certain = []
    volume_certain_correct = []
    volume_certain_incorrect = []
            
    percent_orders_routable = []
    percent_volume_routable = []
    percent_correct_routed_orders = []
    percent_correct_routed_volume = []

    #Final delivery solution
    df_final_solution = pd.read_csv(file_dir + "//" + [file for file in files if dat in file if "Base" in file if "1900" in file][0])
    df_final_solution['SHIPMENTNUMBER'] = df_final_solution['SHIPMENTNUMBER'].apply(ast.literal_eval)
    df_final_solution = df_final_solution.explode("SHIPMENTNUMBER")
    df_final_solution.set_index("SHIPMENTNUMBER", inplace=True)
    df_final_solution = df_final_solution[['originHub', 'idDepot']]
    df_final_solution.rename(columns={"idDepot": "deliveryHub"}, inplace=True)

    #Preliminary delivery solutions
    df = pd.DataFrame(columns=["SHIPMENTNUMBER", "originHub", "idDepot"]).set_index("SHIPMENTNUMBER")
    times = [file.split("_")[2] for file in files if dat in file if "Scenario" in file]
    for time in times:
        print(time)
        scenario_files = [file for file in files if dat in file if time in file if "Scenario" in file][0]
        column_name = "idDepot_" + time.split(".")[0]
        df_temp = pd.read_csv(file_dir + "//" + scenario_files)

        df_temp = df_temp.groupby("SHIPMENTNUMBER").agg({"originHub": "unique",
                                        "idDepot": lambda x: list(x)})
        df_temp[column_name] = df_temp["idDepot"]
        #df_temp['count_unique_idDepot'] = df['unique_idDepot'].apply(lambda x: len(x))
        df =  df.merge(df_temp[[column_name]], left_index=True, right_index=True, how='right')
    df.drop(columns=["originHub", "idDepot"], inplace=True)
    df = df.join(df_temp["originHub"], on="SHIPMENTNUMBER", how='left')
    df.reset_index(inplace=True)
    df["SHIPMENTNUMBER"] = df['SHIPMENTNUMBER'].apply(ast.literal_eval)
    df = df.explode("SHIPMENTNUMBER")
    df = df[df["SHIPMENTNUMBER"] < 5000000000].reset_index(drop=True)

    # add original info and final delivery solution
    df = df.merge(df_free[['SHIPMENTNUMBER', 'idDepot']], on='SHIPMENTNUMBER', how='left')
    df = df.rename(columns={"idDepot": "Free_depot"})
    df = df.merge(df_orders[['SHIPMENTNUMBER', 'PALLETPLAATSEN']], on='SHIPMENTNUMBER', how='left')

    # Apply decission function
    times = [column.split("_")[1] for column in df.columns if "idDepot" in column]
    for time in times:
        df["Choice_" + time] = df.apply(lambda x: majority_element(x["idDepot_" + time], 100), axis=1)
        if time not in ['1000']:
            df["Choice_" + time] = df.apply(lambda x: x["Choice_" + previous_time] if type(x["Choice_" + previous_time]) == int else x["Choice_" + time], axis=1)
        previous_time = time
            

    for time in times:
        total_count.append(len(df[df["Choice_" + time] != "Non Existent"]))
        volume_pallets.append(df[df["Choice_" + time] != "Non Existent"]["PALLETPLAATSEN"].sum())
        
        total_not_certain.append(len(df[df["Choice_" + time] == "Not Certain"]))
        total_certain.append(len(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent")]))
        total_certain_correct.append(len(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] == df["Free_depot"])]))
        total_certain_incorrect.append(len(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] != df["Free_depot"])]))
        
        volume_not_certain.append(df[df["Choice_" + time] == "Not Certain"]["PALLETPLAATSEN"].sum())
        volume_certain.append(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent")]["PALLETPLAATSEN"].sum())
        volume_certain_correct.append(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] == df["Free_depot"])]["PALLETPLAATSEN"].sum())
        volume_certain_incorrect.append(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] != df["Free_depot"])]["PALLETPLAATSEN"].sum())

        percent_orders_routable.append(total_certain[-1] / total_count[-1] * 100)
        percent_volume_routable.append(volume_certain[-1] / volume_pallets[-1] * 100)

        percent_correct_routed_orders.append(total_certain_correct[-1] / total_certain[-1] * 100)
        percent_correct_routed_volume.append(volume_certain_correct[-1] / volume_certain[-1] * 100)
    
    routing_dict[dat] = df

    # # plot the lists
    # times = [column.split("_")[1].replace(".csv", "") for column in df.columns if "idDepot" in column]
    # plt.plot(times, total_count)
    # plt.plot(times, total_not_certain, c='orange')
    # plt.plot(times, total_certain, c='purple')
    # plt.plot(times, total_certain_correct, c='g')
    # plt.plot(times, total_certain_incorrect, c='r')
    # plt.legend(["Total Received Delivery Orders", "Not Certain", "Certain", "Certain correct", "Certain incorrect"])
    # plt.xlabel("Time")
    # plt.xticks(rotation=45)
    # plt.ylabel("Number of orders")
    # plt.title("Number of Orders by Time " + dat)
    # plt.show()

    # plt.plot(times, volume_pallets)
    # plt.plot(times, volume_not_certain, c='orange')
    # plt.plot(times, volume_certain, c='purple')
    # plt.plot(times, volume_certain_correct, c='g')
    # plt.plot(times, volume_certain_incorrect, c='r')
    # plt.legend(["Total Received Delivery Orders", "Uncertain Delivery Depot", "Certain Delivery Depot", "Correct Certain Delivery Depot", "Incorrect Certain Delivery Depot"])
    # plt.xlabel("Time")
    # plt.xticks(rotation=45)
    # plt.ylabel("Volume of orders")
    # plt.title("Volume of orders per time " + dat)
    # plt.show()

    # Plotting the data
    plt.figure(figsize=(10, 6))
    plt.plot(times, percent_orders_routable, label="Percentage Received Orders Routable")
    plt.plot(times, percent_volume_routable, c='orange', label="Percentage Received Volume Routable")
    plt.plot(times, percent_correct_routed_orders, c='purple', label="Percentage Correctly Routed Orders")
    plt.plot(times, percent_correct_routed_volume, c='g', label="Percentage Correctly Routed Volume")

    # Adding labels, legend, and title
    plt.legend(fontsize=14)
    plt.xlabel("Time", fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylabel("Percentage", fontsize=14)
    plt.title("Percentages of Routability", fontsize=16)

    # Display the plot
    plt.tight_layout()
    plt.show()

    # Print some results
    print("orders routable:", percent_orders_routable[-1])
    print("percent_correct_routed_orders:", percent_correct_routed_orders[-1])
    print("volume routable:", percent_volume_routable[-1])
    print("percent_correct_routed_volume:", percent_correct_routed_volume[-1])


    # print("Total orders:           ", total_count)
    # print("Volume pallets:         ", volume_pallets)

    # print("Total not certain:       ", total_not_certain)
    # print("Total certain:           ", total_certain)
    # print("Total certain correct:   ", total_certain_correct)
    # print("Total certain incorrect: ", total_certain_incorrect)

    # print("Volume not certain:      ", volume_not_certain)
    # print("Volume certain:          ", volume_certain)
    # print("Volume certain correct:  ", volume_certain_correct)
    # print("Volume certain incorrect:", volume_certain_incorrect)

    # print("Percent orders routable:       ", percent_orders_routable)
    # print("Percent volume routable:       ", percent_volume_routable)
    # print("Percent correct routed orders: ", percent_correct_routed_orders)
    # print("Percent correct routed volume: ", percent_correct_routed_volume)
        
    

# Single Scenario Approach

In [ ]:
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt

direct = os.getcwd()
file_dir = direct + "\\data\\vos-instances\\Scenario_results\\"
files = os.listdir(file_dir)
dates = list(set([file.split("_")[1] for file in files if "Scenario_" in file]))

# Get final delivery solutions
df_free  = pd.read_csv('totals.csv')
df_free['SHIPMENTNUMBER'] = df_free['SHIPMENTNUMBER'].apply(string_to_int_list)
df_free = df_free.explode("SHIPMENTNUMBER")


file_path = direct + "\\data\\vos_input_data\\MultiHubData3_cleaned.csv" 
df_orders =  pd.read_csv(file_path)

routing_dict_free = {}

for dat in dates:
    total_count = []
    volume_pallets = []
    total_not_certain = []
    total_certain = []
    total_certain_correct = []
    total_certain_incorrect = []

    volume_not_certain = []
    volume_certain = []
    volume_certain_correct = []
    volume_certain_incorrect = []
            
    percent_orders_routable = []
    percent_volume_routable = []
    percent_correct_routed_orders = []
    percent_correct_routed_volume = []

    #Final delivery solution
    df_final_solution = pd.read_csv(file_dir + "//" + [file for file in files if dat in file if "Base" in file if "1900" in file][0])
    df_final_solution['SHIPMENTNUMBER'] = df_final_solution['SHIPMENTNUMBER'].apply(ast.literal_eval)
    df_final_solution = df_final_solution.explode("SHIPMENTNUMBER")
    df_final_solution.set_index("SHIPMENTNUMBER", inplace=True)
    df_final_solution = df_final_solution[['originHub', 'idDepot']]
    df_final_solution.rename(columns={"idDepot": "deliveryHub"}, inplace=True)

    #Preliminary delivery solutions
    df = pd.DataFrame(columns=["SHIPMENTNUMBER", "originHub", "idDepot"]).set_index("SHIPMENTNUMBER")
    times = [file.split("_")[2] for file in files if dat in file if "Scenario" in file]
    for time in times:
        print(time)
        scenario_files = [file for file in files if dat in file if time in file if "Scenario" in file][0]
        column_name = "idDepot_" + time.split(".")[0]
        df_temp = pd.read_csv(file_dir + "//" + scenario_files)

        df_temp = df_temp.groupby("SHIPMENTNUMBER").agg({"originHub": "unique",
                                        "idDepot": lambda x: list(x)})
        df_temp[column_name] = df_temp["idDepot"]
        #df_temp['count_unique_idDepot'] = df['unique_idDepot'].apply(lambda x: len(x))
        df =  df.merge(df_temp[[column_name]], left_index=True, right_index=True, how='right')
    df.drop(columns=["originHub", "idDepot"], inplace=True)
    df = df.join(df_temp["originHub"], on="SHIPMENTNUMBER", how='left')
    df.reset_index(inplace=True)
    df["SHIPMENTNUMBER"] = df['SHIPMENTNUMBER'].apply(ast.literal_eval)
    df = df.explode("SHIPMENTNUMBER")
    df = df[df["SHIPMENTNUMBER"] < 5000000000].reset_index(drop=True)

    # add original info and final delivery solution
    df = df.merge(df_free[['SHIPMENTNUMBER', 'idDepot']], on='SHIPMENTNUMBER', how='left')
    df = df.rename(columns={"idDepot": "Free_depot"})
    df = df.merge(df_orders[['SHIPMENTNUMBER', 'PALLETPLAATSEN']], on='SHIPMENTNUMBER', how='left')

    # Apply decission function
    times = [column.split("_")[1] for column in df.columns if "idDepot" in column]
    for time in times:
        df["Choice_" + time] = df.apply(lambda x: first_item(x["idDepot_" + time]), axis=1)
        if time not in ['1000']:
            df["Choice_" + time] = df.apply(lambda x: int(x["Choice_" + previous_time]) if type(x["Choice_" + previous_time]) == int else x["Choice_" + time], axis=1)
        previous_time = time

    for time in times:
        total_count.append(len(df[df["Choice_" + time] != "Non Existent"]))
        volume_pallets.append(df[df["Choice_" + time] != "Non Existent"]["PALLETPLAATSEN"].sum())
        
        total_not_certain.append(len(df[df["Choice_" + time] == "Not Certain"]))
        total_certain.append(len(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent")]))
        total_certain_correct.append(len(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] == df["Free_depot"])]))
        total_certain_incorrect.append(len(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] != df["Free_depot"])]))
        
        volume_not_certain.append(df[df["Choice_" + time] == "Not Certain"]["PALLETPLAATSEN"].sum())
        volume_certain.append(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent")]["PALLETPLAATSEN"].sum())
        volume_certain_correct.append(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] == df["Free_depot"])]["PALLETPLAATSEN"].sum())
        volume_certain_incorrect.append(df[(df["Choice_" + time] != "Not Certain") & (df["Choice_" + time] != "Non Existent") & (df["Choice_" + time] != df["Free_depot"])]["PALLETPLAATSEN"].sum())

        percent_orders_routable.append(total_certain[-1] / total_count[-1] * 100)
        percent_volume_routable.append(volume_certain[-1] / volume_pallets[-1] * 100)

        percent_correct_routed_orders.append(total_certain_correct[-1] / total_certain[-1] * 100)
        percent_correct_routed_volume.append(volume_certain_correct[-1] / volume_certain[-1] * 100)
    df["Choice_1900"] = df["Choice_1900"].astype(int)
    routing_dict_free[dat] = df

    # plot the lists
    times = [column.split("_")[1].replace(".csv", "") for column in df.columns if "idDepot" in column]
    plt.plot(times, total_count)
    plt.plot(times, total_not_certain, c='orange')
    plt.plot(times, total_certain, c='purple')
    plt.plot(times, total_certain_correct, c='g')
    plt.plot(times, total_certain_incorrect, c='r')
    plt.legend(["Total Received Delivery Orders", "Not Certain", "Certain", "Certain correct", "Certain incorrect"])
    plt.xlabel("Time")
    plt.xticks(rotation=45)
    plt.ylabel("Number of orders")
    plt.title("Number of Orders by Time " + dat)
    plt.show()

    plt.plot(times, volume_pallets)
    plt.plot(times, volume_not_certain, c='orange')
    plt.plot(times, volume_certain, c='purple')
    plt.plot(times, volume_certain_correct, c='g')
    plt.plot(times, volume_certain_incorrect, c='r')
    plt.legend(["Total Received Delivery Orders", "Uncertain Delivery Depot", "Certain Delivery Depot", "Correct Certain Delivery Depot", "Incorrect Certain Delivery Depot"])
    plt.xlabel("Time")
    plt.xticks(rotation=45)
    plt.ylabel("Volume of orders")
    plt.title("Volume of orders per time " + dat)
    plt.show()

    plt.plot(times, percent_orders_routable)
    plt.plot(times, percent_volume_routable, c='orange')
    plt.plot(times, percent_correct_routed_orders, c='purple')
    plt.plot(times, percent_correct_routed_volume, c='g')
    plt.legend(["Percent Orders Routable", "Percent Volume Routable", "Percent Correct Routed Orders", "Percent Correct Routed Volume"])
    plt.xlabel("Time")
    plt.xticks(rotation=45)
    plt.ylabel("Percentage")
    plt.title("Percentages of Routability " + dat)
    plt.show()

    # Print some results

    print(f"{dat} & {percent_orders_routable[-1]:.2f} & {percent_correct_routed_orders[-1]:.2f} & {percent_volume_routable[-1]:.2f} & {percent_correct_routed_volume[-1]:.2f}")
   

    # print("Total orders:           ", total_count)
    # print("Volume pallets:         ", volume_pallets)

    # print("Total not certain:       ", total_not_certain)
    # print("Total certain:           ", total_certain)
    # print("Total certain correct:   ", total_certain_correct)
    # print("Total certain incorrect: ", total_certain_incorrect)

    # print("Volume not certain:      ", volume_not_certain)
    # print("Volume certain:          ", volume_certain)
    # print("Volume certain correct:  ", volume_certain_correct)
    # print("Volume certain incorrect:", volume_certain_incorrect)

    # print("Percent orders routable:       ", percent_orders_routable)
    # print("Percent volume routable:       ", percent_volume_routable)
    # print("Percent correct routed orders: ", percent_correct_routed_orders)
    # print("Percent correct routed volume: ", percent_correct_routed_volume)
        
    

# Create final scenarios to compare cost

In [4]:
depots = {"Woerden": {"depotId": 0,
                      "PC": "NL3447GP"}, 
          "Eindhoven": {"depotId": 1,
                        "PC": "NL5644RL"}, 
          "Beilen": {"depotId": 2,
                     "PC": "NL9411TX"}, 
          "Bornem": {"depotId": 3,
                     "PC": "BE2880"}
          }

units_per_pallet = 100
linehaul_capacity = # ADD LINEHAUL CAPACITY HERE
linehaul_km = # ADD LINEHAUL KM PRICE HERE
crossdock_cost = # ADD CROSSDOCK COST HERE
linehauls = {}

for depot1 in depots:
    for depot2 in depots:
        d1 = depots[depot1]['depotId']
        d2 = depots[depot2]['depotId']
        if d1 < d2:
            linehauls[(d1, d2)] = {"distance": PC_obj.get_distance(depots[depot1]["PC"], depots[depot2]["PC"])}
            linehauls[(d1, d2)]["cost"] = (linehauls[(d1, d2)]["distance"] * (linehaul_km /linehaul_capacity))+ crossdock_cost
        elif d1 > d2:
            linehauls[(d1, d2)] = {"distance": PC_obj.get_distance(depots[depot1]["PC"], depots[depot2]["PC"])}
            linehauls[(d1, d2)]["cost"] = (linehauls[(d2, d1)]["distance"] * (linehaul_km /linehaul_capacity))+ crossdock_cost
        
        else:
            linehauls[(d1, d2)] = {"distance": 0,
                                   "cost": 0}

In [ ]:
from custom.DataCreator import Scenario
path = # PATH TO SOUTIONS HERE
files = os.listdir(path)
files = [files.replace(".csv", "") for files in files if ".csv" in files]
df_free  = pd.read_csv('totals.csv')

# for instance_name in files:
#     print(instance_name)
#     date = instance_name.split("_")[0]
#     if date in routing_dict:
#         df_DSS = routing_dict[date][["SHIPMENTNUMBER", "Choice_1900"]]
#     else:
#         continue
#     scenario = Scenario(path, instance_name, PC_obj)
#     for order in scenario.orders:
#         if order in df_free["SHIPMENTNUMBER"].values:
#             order_num = int(order.replace("[", "").replace("]", "").split(",")[0])
#             if order_num in df_DSS["SHIPMENTNUMBER"].values:
#                 delivery_hub = df_DSS[df_DSS["SHIPMENTNUMBER"] == order_num]["Choice_1900"].values[0]
#             else: 
#                 print(order_num)
#             if type(delivery_hub) == int:
#                 scenario.orders[order]["idDepot"] = delivery_hub
#             else:
#                 scenario.orders[order]["idDepot"] = int(df_free[df_free['SHIPMENTNUMBER'] == order]['idDepot'].values[0])
#         else:
#             print(order)
#     scenario.instance_name = instance_name+"_DSS"
#     print(scenario.instance_name)
#     scenario.write_instance(include_linehauls=False)

for instance_name in files:
    print(instance_name)
    date = instance_name.split("_")[0]
    if date in routing_dict:
        df_SSA = routing_dict_free[date][["SHIPMENTNUMBER", "Choice_1900"]]
    else:
        continue
    scenario = Scenario(path, instance_name, PC_obj)
    for order in scenario.orders:
        if order in df_free["SHIPMENTNUMBER"].values:
            order_num = int(order.replace("[", "").replace("]", "").split(",")[0])
            if order_num in df_SSA["SHIPMENTNUMBER"].values:
                delivery_hub = int(df_SSA[df_SSA["SHIPMENTNUMBER"] == order_num]["Choice_1900"].values[0])
                scenario.orders[order]["idDepot"] = delivery_hub
            else:
                print(order_num)
                scenario.orders[order]["idDepot"] = int(df_free[df_free['SHIPMENTNUMBER'] == order]['idDepot'].values[0])
        else:
            print(order)
    scenario.instance_name = instance_name+"_SSA"
    print(scenario.instance_name)
    scenario.write_instance(include_linehauls=False)

# Calculate linehaul cost


In [ ]:
file_dir = direct + "\\data\\vos-instances\\fixedVSfree\\"
files = os.listdir(file_dir)
files = [files.replace(".csv", "") for files in files if ".csv" in files]
files

In [ ]:
df_free  = pd.read_csv('totals.csv')

def return_DSS_hub(row):
    shipnr = int(row["SHIPMENTNUMBER"].replace("[", "").replace("]", "").split(",")[0])
    if shipnr in df_DSS["SHIPMENTNUMBER"].values:
        hub = df_DSS[df_DSS["SHIPMENTNUMBER"] == shipnr]["Choice_1900"].values[0]
        if type(hub) == int:
            return int(hub)
        else:
            return row["Free_idDepot"]
    else:
        row["Free_idDepot"]

def return_SSA_hub(row):
    shipnr = int(row["SHIPMENTNUMBER"].replace("[", "").replace("]", "").split(",")[0])
    if shipnr in df_SSA["SHIPMENTNUMBER"].values:
        hub = df_SSA[df_SSA["SHIPMENTNUMBER"] == shipnr]["Choice_1900"].values[0]
        return int(hub)
    else:
        return row["Free_idDepot"]


path = # PATH TO SOLUTION FILES HERE
files = os.listdir(path)
files = [files.replace(".csv", "") for files in files if ".csv" in files]

for file in files:
    print(file)
    date = file.split("_")[0]
    if date in routing_dict:
        df_DSS = routing_dict[date][["SHIPMENTNUMBER", "Choice_1900"]]
        df_SSA = routing_dict_free[date][["SHIPMENTNUMBER", "Choice_1900"]]
    df_scenario = pd.read_csv(path + file + ".csv")
    df_scenario['origin_idDepot_CPC'] = df_scenario.apply(lambda x: PC_obj.get_closest_hub(x["LAAD_CPC"], [depots[depot]['PC'] for depot in depots]), axis=1)
    df_scenario['origin_idDepot'] = df_scenario.apply(lambda x: [depots[depot]['depotId'] for depot in depots if depots[depot]['PC'] == x['origin_idDepot_CPC']][0], axis=1)

    df_scenario = df_scenario.merge(df_free[['SHIPMENTNUMBER', 'idDepot']], on='SHIPMENTNUMBER', how='left')
    df_scenario.rename(columns={"idDepot": "Free_idDepot"}, inplace=True)


    df_scenario["DSS_idDepot"] = df_scenario.apply(return_DSS_hub, axis=1)
    df_scenario.loc[df_scenario['DSS_idDepot'].isna(), 'DSS_idDepot'] = df_scenario['Free_idDepot']

    df_scenario['SSA_idDepot'] = df_scenario.apply(return_SSA_hub, axis=1)

    df_scenario['origin_idDepot_CPC'] = df_scenario.apply(lambda x: PC_obj.get_closest_hub(x["LAAD_CPC"], [depots[depot]['PC'] for depot in depots]), axis=1)
    df_scenario['origin_idDepot'] = df_scenario.apply(lambda x: [depots[depot]['depotId'] for depot in depots if depots[depot]['PC'] == x['origin_idDepot_CPC']][0], axis=1)

    # get the free iddepot from the corresponding entry in df
    df_scenario = df_scenario[~df_scenario['Free_idDepot'].isna()]
    df_scenario['LinehaulCostDSS'] = df_scenario.apply(lambda x: linehauls[(x['origin_idDepot'], x['DSS_idDepot'])]["cost"] * x['PALLETPLAATSEN'] , axis=1)
    # df_scenario['Linehauled'] = df_scenario.apply(lambda x: 1 if x['idDepot'] != x['Fixed_idDepot'] else 0, axis=1)
    total_cost = df_scenario['LinehaulCostDSS'].sum()
    print(f"Total DSS cost for {file} is {total_cost}")

    df_scenario['LinehaulCostSSA'] = df_scenario.apply(lambda x: linehauls[(x['origin_idDepot'], x['SSA_idDepot'])]["cost"] * x['PALLETPLAATSEN'] , axis=1)
    # df_scenario['Linehauled'] = df_scenario.apply(lambda x: 1 if x['idDepot'] != x['Fixed_idDepot'] else 0, axis=1)
    total_cost = df_scenario['LinehaulCostSSA'].sum()
    print(f"Total SSA cost for {file} is {total_cost}")
